__This code creates a random distribution of atoms inside a box subject to the PBC__


In [5]:
import periodictable

def get_atomic_masses(atom_labels):
    '''Input: list of atomic symbols of different species of atoms
    Outputs: atomic masses of the atom types
    '''
    atomic_masses = []
    unsupported_labels = []
    
    for label in atom_labels:
        try:
            element = getattr(periodictable, label)
            atomic_mass = element.mass * 1.66053904020e-24 # Atomic mass in gm
            atomic_masses.append(atomic_mass)
        except AttributeError:
            unsupported_labels.append(label)
    if unsupported_labels:
        print(f"Unsupported atomic labels: {', '.join(unsupported_labels)}")
    
    return atomic_masses

# List of atomic labels for our system
atomic_labels = ['Li', 'Nb', 'O']

# Get the corresponding atomic masses
atomic_masses = get_atomic_masses(atomic_labels)

In [6]:
import numpy as np
# Create a list of number of atoms of each species
# Order must be same as in atomic_labels

# N_sp = [40000, 40000, 120000]
N_sp = [5000, 5000, 15000]
rho = 4.39                      # density of the material in gm/cc. This is what you should know. 

total_mass = sum(x * y for x, y in zip(atomic_masses, N_sp))
vol_in_A3 = (total_mass/rho)*1e24

box = np.cbrt(vol_in_A3)
print(f"The box size is {box} Angstorm.")

The box size is 65.39145086689086 Angstorm.


In [7]:
def periodic_distance(atom, nbd, box):
    '''Inputs:
        atom = coordinate of a single reference atom
        nbd = array of coordinates of multiple atoms
        box = length of the box
       Outputs:
        dis = distance between atom and nbd
        len(dis) == len(nbd)
    '''
    delta = np.abs(atom-nbd)
    delta = np.where(delta> 0.5*box, delta-box, delta)
    dis = np.sqrt((delta**2).sum(axis=-1))
    return dis

In [9]:
from tqdm import tqdm
import random as ran
import numpy as np


cut_off = [1.8, 1.5, 1.3 ]                #minimum approach distance
cum_N = [sum(N_sp[:i + 1]) for i in range(len(N_sp))]  # Cumulative N_Sp 
rx,ry,rz = [],[],[]
print ("Now creating the random coordinates.................")

for ind in range(len(atomic_labels)):
    print (f"Now placing {atomic_labels[ind]}!!!")
    for count in tqdm(range(cum_N[ind]), desc="Processing", ncols=100):
        x,y,z = box*ran.random(), box*ran.random(), box*ran.random()
        checker = "Failed"
        while checker != "Passed":
            dist = periodic_distance([x,y,z], np.array([rx,ry,rz]).T, box)
            if all(value > cut_off[ind] for value in dist):
                checker = "Passed"
            else:
                x,y,z = box*ran.random(), box*ran.random(), box*ran.random()
        rx.append(x)
        ry.append(y)
        rz.append(z)


Now creating the random coordinates.................
Now placing Li!!!


Processing: 100%|█████████████████████████████████████████████| 5000/5000 [00:04<00:00, 1191.55it/s]


Now placing Nb!!!


Processing: 100%|████████████████████████████████████████████| 10000/10000 [00:44<00:00, 226.82it/s]


Now placing O!!!


Processing: 100%|█████████████████████████████████████████████| 25000/25000 [12:10<00:00, 34.22it/s]


__Great!!!__
We now have the coordinates. Let us save it in some suitable format for running simulations. 
For now we use the VASP format for this coordinates we have generated.

In [10]:
out1 = open("25k_random.POSCAR", 'w')

print ("comment line",file=out1)
print ("%16.10f"%(1.0),file=out1)
print ("%23.16f"%(box),"%22.16f"%(0.0),"%22.16f"%(0.0),file=out1)
print ("%23.16f"%(0.0),"%22.16f"%(box),"%22.16f"%(0.0),file=out1)
print ("%23.16f"%(0.0),"%22.16f"%(0.0),"%22.16f"%(box),file=out1)
print ('\t'.join(atomic_labels),file=out1)
print ('\t'.join(map(str, N_sp)),file=out1)
print ("Cartesian",file=out1)

for ct in range(len(rx)):
        print ("%20.16f"%(rx[ct]),"%20.16f"%(ry[ct]),"%20.16f"%(rz[ct]),file=out1)
        
out1.close()
